In [3]:
from shap import KernelExplainer
import shap
import matplotlib.pyplot as plt 
%matplotlib inline

import joblib
import numpy as np 
from pypal.models.gpr import predict_coregionalized
from dispersant_screener.utils import read_pickle, get_summary_stats_time
from dispersant_screener.definitions import FEATURES

from sklearn.preprocessing import StandardScaler
import pickle
import pandas as pd
DATADIR = '../data'
import os 

from functools import partial

In [4]:
def load_data(n_samples, label_scaling: bool = False):
    """Take in Brian's data and spit out some numpy arrays for the PAL"""
    df_full_factorial_feat = pd.read_csv(os.path.join(DATADIR, 'new_features_full_random.csv'))[FEATURES].values
    a2 = pd.read_csv(os.path.join(DATADIR, 'b1-b21_random_virial_large_new.csv'))['A2_normalized'].values
    deltaGMax = pd.read_csv(os.path.join(DATADIR, 'b1-b21_random_virial_large_new.csv'))['A2_normalized'].values  # pylint:disable=unused-variable
    gibbs = pd.read_csv(os.path.join(DATADIR, 'b1-b21_random_deltaG.csv'))['deltaGmin'].values * (-1)
    gibbs_max = pd.read_csv(os.path.join(DATADIR, 'b1-b21_random_virial_large_new.csv'))['deltaGmax'].values
    force_max = pd.read_csv(os.path.join(DATADIR, 'b1-b21_random_virial_large_fit2.csv'))['F_repel_max'].values  # pylint:disable=unused-variable
    rg = pd.read_csv(os.path.join(DATADIR, 'rg_results.csv'))['Rg'].values
    y = np.hstack([rg.reshape(-1, 1), gibbs.reshape(-1, 1), gibbs_max.reshape(-1, 1)])
    assert len(df_full_factorial_feat) == len(a2) == len(gibbs) == len(y)

    feat_scaler = StandardScaler()
    X = feat_scaler.fit_transform(df_full_factorial_feat)

    if label_scaling:
        label_scaler = MinMaxScaler()
        y = label_scaler.fit_transform(y)

    #greedy_indices = get_maxmin_samples(X, n_samples)

    return X, y#, greedy_indice

In [5]:
X, y = load_data(1)

In [6]:
with open('../work/sweeps3/20201021-235927_dispersant_0.05_0.05_0.1111111111111111_60-models.pkl', 'rb') as fh:
    models = pickle.load(fh)

In [7]:
sampled = np.load('../work/sweeps3/20201021-235927_dispersant_0.05_0.05_0.1111111111111111_60-selected.npy', allow_pickle=True)

In [8]:
sampled_indices = sampled[-1]

In [9]:
def model_wrapper(X, i=0): 
    mu, var = predict_coregionalized(models[0], X, i)
    return mu.flatten()

In [12]:
shap0 = KernelExplainer(partial(model_wrapper, i=0), shap.kmeans(X[sampled_indices], 50))

In [13]:
shap0_values = shap0.shap_values(X)

 /Users/kevinmaikjablonka/opt/miniconda3/envs/dispersant_basf/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning:l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
 /Users/kevinmaikjablonka/opt/miniconda3/envs/dispersant_basf/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning:l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
 /Users/kevinmaikjablonka/opt/miniconda3/envs/dispersant_basf/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning:l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
 /Users/kevinmaikjablonka/opt/miniconda3/envs/dispersant_basf/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning:l1_reg="auto" is deprecated and i

 /Users/kevinmaikjablonka/opt/miniconda3/envs/dispersant_basf/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning:l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!


In [14]:
shap1 = KernelExplainer(partial(model_wrapper, i=1),  shap.kmeans(X[sampled_indices], 50))

In [15]:
shap1_values = shap1.shap_values(X)

 /Users/kevinmaikjablonka/opt/miniconda3/envs/dispersant_basf/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning:l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
 /Users/kevinmaikjablonka/opt/miniconda3/envs/dispersant_basf/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning:l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
 /Users/kevinmaikjablonka/opt/miniconda3/envs/dispersant_basf/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning:l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
 /Users/kevinmaikjablonka/opt/miniconda3/envs/dispersant_basf/lib/python3.7/site-packages/shap/explainers/kernel.py:555: UserWarning:l1_reg="auto" is deprecated and i

KeyboardInterrupt: 

In [ ]:
shap2 = KernelExplainer(partial(model_wrapper, i=2),  shap.kmeans(X[sampled_indices], 50))

In [ ]:
shap2_values = shap2.shap_values(X)

In [15]:
np.save('shap0_latest.npy', shap0_values)
np.save('shap1_latest.npy', shap1_values)
np.save('shap2_latest.npy', shap2_values)

In [44]:
FEATURES

['num_[W]',
 'max_[W]',
 'num_[Tr]',
 'max_[Tr]',
 'num_[Ta]',
 'max_[Ta]',
 'num_[R]',
 'max_[R]',
 '[W]',
 '[Tr]',
 '[Ta]',
 '[R]',
 'rel_shannon',
 'length']

In [16]:
import shap

In [ ]:
shap.summary_plot(shap0_values, X, 
                  plot_type='layered_violin', feature_names=FEATURES, plot_size=(4,6),
                 show=False, max_display=6, color='seismic'
                 )
#plt.savefig('shap_target_0.pdf', bbox_inches='tight')

In [ ]:
shap.summary_plot(shap1_values, X, 
                  plot_type='layered_violin', feature_names=FEATURES,
                 plot_size=(4,6), max_display=6,
                  show=False, color='seismic'
                 )

#plt.savefig('shap_target_1.pdf', bbox_inches='tight')

In [ ]:
shap.summary_plot(shap2_values, X, 
                  plot_type='layered_violin', feature_names=FEATURES,
                 plot_size=(4,6), max_display=10, color='seismic',
                  show=False
                 )

#plt.savefig('shap_target_2.pdf', bbox_inches='tight')